In [19]:
import json
import os
import numpy as np
from tensorflow import keras
from sensorflow_model.dataset import get_dataset


In [20]:
def normalize_with_saved_stats(X, stats_path="../normalization.json"):
    with open(stats_path) as f:
        stats = json.load(f)

    mean = np.array(stats["mean"])
    std = np.array(stats["std"])

    # Apply normalization
    X_flat = X.reshape(-1, X.shape[-1])
    X_norm = ((X_flat - mean) / std).reshape(X.shape)
    return X_norm


In [21]:
from sensorflow_model.dataset import create_sliding_windows, load_csv, normalize_features


def get_dataset(filepath, window_size=20, stride=5, normalize=True, use_saved_stats=False, stats_path="../normalization.json"):
    df = load_csv(filepath)
    X, y = create_sliding_windows(df, window_size, stride)

    if normalize:
        if use_saved_stats:
            X = normalize_with_saved_stats(X, stats_path)
        else:
            X, _ = normalize_features(X)

    return X, y


In [22]:
def load_test_dataset(folder_path):
    X_all, y_all = [], []

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            filepath = os.path.join(folder_path, file)

            # Load with test-time normalization (using training stats)
            X, y = get_dataset(filepath, window_size=20, stride=5, normalize=True, use_saved_stats=True)

            X_all.append(X)
            y_all.append(y)

    return np.concatenate(X_all), np.concatenate(y_all)


In [23]:

# Step 1: Load model
model = keras.models.load_model("../final_model.keras")


c:\Users\mohit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 46 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
# Step 2: Load test data
x_test, y_test = load_test_dataset("../TestData")

In [25]:

# Step 3: Evaluate
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"\n Test Accuracy: {accuracy * 100:.2f}%")

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0078 - loss: 6.3882 

 Test Accuracy: 1.59%


In [26]:

ACTIVITY_CLASSES = {
    0: "Sitting + Typing on Desk",
    1: "Sitting + Taking Notes", 
    2: "Standing + Writing on Whiteboard",
    3: "Standing + Erasing Whiteboard",
    4: "Sitting + Talking + Waving Hands",
    5: "Standing + Talking + Waving Hands",
    6: "Sitting + HeadNodding",
    7: "Sitting + Drinking Water",
    8: "Sitting + Drinking Coffee",
    9: "Standing + Drinking Water",
    10: "Standing + Drinking Coffee",
    11: "Scrolling on Phone",
}


In [27]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)

labels = list(ACTIVITY_CLASSES.keys())
target_names = list(ACTIVITY_CLASSES.values())

print(classification_report(y_test, y_pred_labels, labels=labels, target_names=target_names))


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
                                   precision    recall  f1-score   support

         Sitting + Typing on Desk       0.00      0.00      0.00       239
           Sitting + Taking Notes       0.00      0.00      0.00       115
 Standing + Writing on Whiteboard       0.00      0.00      0.00         0
    Standing + Erasing Whiteboard       0.03      0.06      0.04       263
 Sitting + Talking + Waving Hands       0.00      0.00      0.00         0
Standing + Talking + Waving Hands       0.00      0.00      0.00       241
            Sitting + HeadNodding       0.00      0.00      0.00         0
         Sitting + Drinking Water       0.00      0.01      0.01       189
        Sitting + Drinking Coffee       0.00      0.00      0.00       271
        Standing + Drinking Water       0.00      0.00      0.00       139
       Standing + Drinking Coffee       0.14      0.06      0.09       161
               Scrolling on Phone       0.01      0.00      

c:\Users\mohit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\mohit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\mohit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver